### CNN on CIFR Assignment:

1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
3.  You cannot use DropOut layers.
4.  You MUST use Image Augmentation Techniques.
5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
7.  You cannot use test images for training the model.
8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
10. You cannot have more than 1 Million parameters in total
11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
12. You can use any optimization algorithm you need. 
13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [1]:
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten, Input
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
num_classes= 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

In [3]:
X_train.shape

(50000, 32, 32, 3)

In [4]:
X_test.shape

(10000, 32, 32, 3)

In [5]:
y_test.shape

(10000, 10)

In [6]:
def normalize_data(X):
    return (X - np.mean(X))/np.std(X)

In [7]:
X_train = normalize_data(X_train.astype("float32"))
X_test = normalize_data(X_test.astype("float32"))

In [8]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs =200
l = 12
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [9]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [10]:
def dense_model(input_shape = (32,32,3), n_classes = 10):
    X_input = Input(shape=(img_height, img_width, channel))
    First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(X_input)
    
    First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
    First_Transition = transition(First_Block, num_filter, dropout_rate)

    Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
    Second_Transition = transition(Second_Block, num_filter, dropout_rate)

    Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
    Third_Transition = transition(Third_Block, num_filter, dropout_rate)

    Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
    output = output_layer(Last_Block)
    
    model = Model(inputs=X_input, outputs=output)
    return model

In [11]:
model = dense_model(input_shape = (32,32,3), n_classes = 10)

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 12)   48          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 12)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [13]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [14]:
given_batch_size = 1
given_step_size = 4
steps = int(X_train.shape[0] / given_step_size)
datagen = ImageDataGenerator()
train_data = datagen.flow(X_train, y_train, batch_size=given_batch_size)
valid_data = datagen.flow(X_test, y_test, batch_size=given_batch_size)

In [ ]:
model.fit(train_data,
                    steps_per_epoch=steps,
                    epochs=epochs,
                    verbose=1, 
                    validation_data=valid_data)

Epoch 1/200
